# Vocalization analysis in live

TODO
- Titles / graduation etc
- same color everywere for f0
- understand the convolution

## Import

In [1]:
import os
import math
import glob
import librosa
import time
import libf0
import statistics
import numpy as np
import pandas as pd
from scipy import signal
from skimage.restoration import denoise_tv_chambolle
from scipy.io import wavfile
import matplotlib.pyplot as plt
from matplotlib import gridspec

from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets

C:\Users\de Witasse Thézy\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## Trigger and save by Avisoft

## Import some data

In [2]:
basepath = "D:\\data\\playback2T01\\ch2"
listRecordingsPath = glob.glob(os.path.join(basepath, '*.wav'))

# Analysis step by step with interactive graphs

## Compute the fundamental frequency

parameters :
With the widgets, you can make two parameters change :
- the minmum fundamental frequency (fmin)
- the hop leght (H), which is the size of the non overlapping part of the window

In [15]:
def compute_and_show_f0(VocNumber, h, fmin, returnf0) :
    y, sr = librosa.load(listRecordingsPath[VocNumber], sr = 300000)
    
    f0,_,_= libf0.salience(y, Fs=sr, N=500, H=h, F_min=fmin, F_max=90000, R=10.0, num_harm=10,
        freq_smooth_len=11, alpha=0.9, gamma=0.0, constraint_region=None, tol=5, score_low=0.01, score_high=1.0)
    times = librosa.times_like(f0, sr = sr,  hop_length = h)

    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)

    fig, ax = plt.subplots()

    img = librosa.display.specshow(D, x_axis='ms', y_axis='linear', ax=ax,  sr=sr, fmin=20000, fmax=100000)

    #ax.set(title='pYIN fundamental frequency estimation, duration = %f'%(duration))

    ax.set_ylim(20000, 100000)

    fig.colorbar(img, ax=ax, format="%+2.f dB")

    ax.plot(times, f0, label='f0 extimated with libf0', color='cyan', linewidth=3)

    ax.legend(loc='upper right')
    
    if returnf0 :
        return f0,times

interact(compute_and_show_f0, VocNumber = 4, h=IntSlider(min=50, max=1000, step=50, value=600), 
         fmin = IntSlider(min=20000, max=50000, step=2000, value=38000),returnf0 = fixed(False))

interactive(children=(IntSlider(value=4, description='VocNumber', max=12, min=-4), IntSlider(value=600, descri…

<function __main__.compute_and_show_f0(VocNumber, h, fmin, returnf0)>

## extract the vocalization

In [10]:
def find_and_show_steps(data, width, plot=False):
    dary = np.array(data)
    dary -= np.average(dary)
    step = np.hstack((np.ones(len(dary)), -1*np.ones(len(dary))))
    dary_step = np.convolve(dary, step, mode='valid')
    # Get the peaks of the convolution
    peaks = signal.find_peaks(dary_step, width=width)[0]
    
    # plots
    if plot :
        plt.plot(dary)
        plt.plot(dary_step/10)
        for ii in range(len(peaks)):
             plt.plot((peaks[ii], peaks[ii]), (min(min(dary_step/10), min(dary)) -2,max(max(dary),max(dary_step/10))+2), 'r')
        plt.ylim([min(min(dary_step/10), min(dary))-2,max(max(dary),max(dary_step/10))+2])
        plt.show()
    
    return peaks

def find_voc(VocNumber, weight, width, plot, h = 500, returnSteps = True) :
    f0,times = compute_and_show_f0(VocNumber=VocNumber, h=h,fmin=38000, returnf0 = True)
    plt.close()
    x = np.array(f0)
    #plt.plot(f0)
    x_std = (x - x.mean()) / x.std()
    x_denoiseP = (denoise_tv_chambolle(x_std, weight= weight))**2
    x_denoiseN = -(denoise_tv_chambolle(x_std, weight= weight))**2
    
    stepsP = find_and_show_steps(x_denoiseP, width, plot)
    stepsN = find_and_show_steps(x_denoiseN, width, plot)
    
    if returnSteps :
        return [stepsP, stepsN,f0,times]
    
    
interact(find_voc, VocNumber=4, weight=0.5, 
        width=IntSlider(min=2, max=30, step=2, value=10),
        plot = fixed(True),
         h=fixed(600),
        returnSteps = fixed(False))

interactive(children=(IntSlider(value=4, description='VocNumber', max=12, min=-4), FloatSlider(value=0.5, desc…

<function __main__.find_voc(VocNumber, weight, width, plot, h=500, returnSteps=True)>

In [12]:
def extract_vocalizationf0(VocNumber, noise_threshold, plot, h = 500):
    stepsP, stepsN, f0, times = find_voc(VocNumber, weight = 0.5, width = 10, h = h, plot = False)
    vocf0 = []
    voctime = []
    vocduration = [0]
    for i in range(min(len(stepsP), len(stepsN))):
        if np.mean(f0[stepsP[i]:stepsN[i]]) < noise_threshold :
            continue
        vocf0.append(list(f0[stepsP[i]:stepsN[i]]))
        voctime.append(list(times[stepsP[i]:stepsN[i]]))
        vocduration.append((times[stepsN[i]]) - times[stepsP[i]])

    Name = listRecordingsPath[VocNumber].split("_")[-1].split(".")[0]
    
    y, sr = librosa.load(listRecordingsPath[VocNumber], sr = 300000)
    if plot :
        plt.figure(figsize = (10,6))
        ax = plt.gca()
        D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
        img = librosa.display.specshow(D, x_axis='ms', y_axis='linear', ax=ax,  sr=sr, fmin=20000, fmax=100000)
        ax.set(title='fundamental frequency estimation,\n duration estimation = %f ms'%(vocduration[-1]))
        ax.set_ylim(20000, 100000)
        plt.colorbar(img, ax=ax, format="%+2.f dB")
        for vocnumber in range(len(vocf0)) :
            ax.plot(voctime[vocnumber], vocf0[vocnumber], label='f0', color='cyan', linewidth=2, linestyle = '-')
        ax.legend(loc='upper right')
       
interact(extract_vocalizationf0, VocNumber=4, noise_threshold=IntSlider(min=20000, max=70000, step=5000, value=45000), plot = fixed(True), 
         h=IntSlider(min=50, max=1200, step=50, value=500))

interactive(children=(IntSlider(value=4, description='VocNumber', max=12, min=-4), IntSlider(value=45000, desc…

<function __main__.extract_vocalizationf0(VocNumber, noise_threshold, plot, h=500)>

# The general function

In [263]:
def ExtractRecordingsPaths(playback_time_str, basepath) :
    """return a list of wavefiles paths"""
    # TODO : check that it's working
    playback_time = time.strptime(playback_time_str, '%H-%M-%S')
    listRecordingsPaths = glob.glob(os.path.join(basepath, '*.wav'))
    listgoodtime = []
    for RecordingPath in listRecordingsPaths :
        recording_time_str = RecordingPath.split('_')[-2]
        recording_time = time.strptime(recording_time_str, '%H-%M-%S')
        if playback_time < recording_time : #on peut aussi ajouter un seuil pour que ce ne soit pas trop longtemps après ??
            listgoodtime.append(RecordingPath)
    return listgoodtime

In [264]:
def find_steps(data) :
    dary = np.array(data)
    dary -= np.average(dary)
    step = np.hstack((np.ones(len(dary)), -1*np.ones(len(dary))))
    dary_step = np.convolve(dary, step, mode='valid')
    # Get the peaks of the convolution
    peaks = signal.find_peaks(dary_step, width=10)[0]
    return peaks


def process_single_audio(file_path, h, completef0 = False):

    try:
        y, sr = librosa.load(file_path, sr = 300000) # important, garder cette frequence d'echantillognage, sinon faux.

        #y = nr.reduce_noise(y1, sr, hop_length = 512, freq_mask_smooth_hz = 1000)
        f0,_,_= libf0.salience(y, Fs=sr, N=500, H=h, F_min=40000, F_max=90000, R=10.0, num_harm=10, freq_smooth_len=11, alpha=0.9, gamma=0.0, constraint_region=None, tol=5, score_low=0.01, score_high=1.0)
        times = librosa.times_like(f0, sr = sr,  hop_length = h)

        #very depend of the F thresholds

        # Convolution part
        # denoising the extimated f0
        x = np.array(f0)
        x_std = (x - x.mean()) / x.std()
        x_denoiseP = (denoise_tv_chambolle(x_std, weight= 0.5))**2
        x_denoiseN = -(denoise_tv_chambolle(x_std, weight=0.5))**2
        # find the steps
        stepsP = find_steps(x_denoiseP)
        stepsN = find_steps(x_denoiseN)

        vocf0 = []
        voctime = []
        vocduration = []
        voctotf0 = []
        # for each vocalization found in the audio
        for i in range(min(len(stepsP), len(stepsN))):
            if np.mean(f0[stepsP[i]:stepsN[i]]) < 45000 :
                continue
            vocf0.append(np.mean(f0[stepsP[i]:stepsN[i]]))
            voctotf0.append(f0[stepsP[i]:stepsN[i]])
            voctime.append(list(times[stepsP[i]:stepsN[i]]))
            vocduration.append(times[stepsN[i]] - times[stepsP[i]])
        if not vocf0 :
            return None
        if completef0 :
            return {
            'Filename': [file_path.split("_")[-1].split(".")[0]]*len(vocf0),
            'F0' : voctotf0,
            'Duration' : voctime
        }
        return {
            'Filename': [file_path.split("_")[-1].split(".")[0]]*len(vocf0),
            'F0' : vocf0,
            'Duration' : vocduration
        }
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [265]:
def ExtractFeaturesRecordings(file_paths, h):
    results = {
            'Filename': [],
            'Frequency' : [],
            'Duration' : [],
        }

    for file_path in file_paths:
        result = process_single_audio(file_path, h)
        if result is not None:
            results['Filename'].extend(result['Filename'])
            results['Frequency'].extend(result['F0'])
            results['Duration'].extend(result['Duration'])

    df = pd.DataFrame(results)
    df = df.dropna(axis=0, how='any')
    return df

In [266]:
def VocAnalysis(playback_time_str, basepath, Feature = "VocNumber", thresholds = [0, 100], h=500) :
    # TODO : check for method error. (try, except)
    listRecordingsPaths = ExtractRecordingsPaths(playback_time_str, basepath)
    if len(listRecordingsPaths) == 0 :
        return False
    FeaturesDF = ExtractFeaturesRecordings(listRecordingsPaths, h)
    if Feature == "VocNumber" :
        if (len(FeaturesDF) >= min(thresholds) and len(FeaturesDF) < max(thresholds)) :
            return True
    elif Feature == "Duration" or Feature == "Frequency":
        if any(FeaturesDF[Feature] >= min(thresholds)) and any(FeaturesDF[Feature] < max(thresholds)) :
            return True
    return False

In [267]:
playback_time_str = '15-41-48'

In [268]:
start = time.perf_counter()

print(VocAnalysis(playback_time_str, basepath, Feature = "Frequency", thresholds = [60000, 70000]))

finish = time.perf_counter()
print(f'It took {finish-start:.2f} second(s) to finish')

C:\ProgramData\Anaconda3\lib\site-packages\libf0\salience.py:440: RuntimeWarning: invalid value encountered in cast
  bin_index = np.floor((1200 / R) * np.log2(F / F_ref) + 0.5).astype(np.int64)


True
It took 0.78 second(s) to finish
